In [ ]:
#be careful, the use of twint is against the User Service Agreement of Twitter, this is only an experiment

##Download Tweets

In [ ]:
!pip3 install twint

In [ ]:
https://github.com/twintproject/twint

In [ ]:
import twint

# Configure
c = twint.Config()
c.Search = '#thanos', 'Thanos'
#c.Search = 'Thanos', 'snap'
c.Store_csv = True
c.Since = "2018-06-16 00:00:00" #anno, mese, giorno
#c.g="48.880048,2.385939,1km" #1km radius from Paris
c.Limit = 10000 #non funziona
c.Output = "Thanos.csv"

# Run
twint.run.Search(c)

##Install torchMoji

In [3]:
!pip3 install torch==1.0.1 -f https://download.pytorch.org/whl/cpu/stable 
!git clone https://github.com/huggingface/torchMoji
import os
os.chdir('torchMoji')
!pip3 install -e .
#if you restart the package, the notebook risks to crash on a loop
#if you managed to be curious and to make it stuck, just clic on RunTime, Factory Reset Runtime
#I did not restart and worked fine

#se questo funziona, poi crasha in future linee di codice, anche se chiudiamo e riapriamo dovrebbe essere a posto per 12 ore

Looking in links: https://download.pytorch.org/whl/cpu/stable
     |████████████████████████████████| 560.1MB 28kB/s 
ERROR: torchvision 0.6.0+cu101 has requirement torch==1.5.0, but you'll have torch 1.0.1 which is incompatible.
  Found existing installation: torch 1.5.0+cu101
    Uninstalling torch-1.5.0+cu101:
      Successfully uninstalled torch-1.5.0+cu101
Cloning into 'torchMoji'...
remote: Enumerating objects: 143, done.
remote: Total 143 (delta 0), reused 0 (delta 0), pack-reused 143
Receiving objects: 100% (143/143), 2.41 MiB | 4.19 MiB/s, done.
Resolving deltas: 100% (49/49), done.
Obtaining file:///content/torchMoji
     |████████████████████████████████| 17.0MB 231kB/s 
     |████████████████████████████████| 48.2MB 81kB/s 
     |████████████████████████████████| 12.4MB 18.9MB/s 
     |████████████████████████████████| 81kB 9.0MB/s 
  Created wheel for emoji: filename=emoji-0.4.5-cp36-none-any.whl size=38198 sha256=fd400b14a45278ab313ca42ecfe001bd02fb8b957628a1367b0786cdd65

In [4]:
!python3 scripts/download_weights.py

About to download the pretrained weights file from https://www.dropbox.com/s/q8lax9ary32c7t9/pytorch_model.bin?dl=0#
The size of the file is roughly 85MB. Continue? [y/n]
y
Downloading...
Running system call: wget https://www.dropbox.com/s/q8lax9ary32c7t9/pytorch_model.bin?dl=0# -O /content/torchMoji/model/pytorch_model.bin
--2020-06-21 20:19:30--  https://www.dropbox.com/s/q8lax9ary32c7t9/pytorch_model.bin?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/q8lax9ary32c7t9/pytorch_model.bin [following]
--2020-06-21 20:19:30--  https://www.dropbox.com/s/raw/q8lax9ary32c7t9/pytorch_model.bin
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc514f4c8f73ad2aebba32309a8e.dl.dropboxusercontent.com/cd/0/inline/A6EhM8RknHW0cw9

In [1]:
#si connette a DeepMoji per una request, non posso modificare i parametri, credo
!python3 examples/text_emojize.py --text f" {Stay safe from the virus} "

python3: can't open file 'examples/text_emojize.py': [Errno 2] No such file or directory


In [5]:
import numpy as np
import emoji, json
from torchmoji.global_variables import PRETRAINED_PATH, VOCAB_PATH
from torchmoji.sentence_tokenizer import SentenceTokenizer
from torchmoji.model_def import torchmoji_emojis
  
EMOJIS = ":joy: :unamused: :weary: :sob: :heart_eyes: :pensive: :ok_hand: :blush: :heart: :smirk: :grin: :notes: :flushed: :100: :sleeping: :relieved: :relaxed: :raised_hands: :two_hearts: :expressionless: :sweat_smile: :pray: :confused: :kissing_heart: :heartbeat: :neutral_face: :information_desk_person: :disappointed: :see_no_evil: :tired_face: :v: :sunglasses: :rage: :thumbsup: :cry: :sleepy: :yum: :triumph: :hand: :mask: :clap: :eyes: :gun: :persevere: :smiling_imp: :sweat: :broken_heart: :yellow_heart: :musical_note: :speak_no_evil: :wink: :skull: :confounded: :smile: :stuck_out_tongue_winking_eye: :angry: :no_good: :muscle: :facepunch: :purple_heart: :sparkling_heart: :blue_heart: :grimacing: :sparkles:".split(' ')
model = torchmoji_emojis(PRETRAINED_PATH)
with open(VOCAB_PATH, 'r') as f:
  vocabulary = json.load(f)
st = SentenceTokenizer(vocabulary, 30)

def deepmojify(sentence, top_n=5, return_emoji=True, return_prob=False):
  #converte lista probabilità in emoticon più probabili
  def top_elements(array, k):
    ind = np.argpartition(array, -k)[-k:]
    return ind[np.argsort(array[ind])][::-1]

  tokenized, _, _ = st.tokenize_sentences([sentence])
  #print(tokenized)
  #lista di probabilità
  prob = model(tokenized)[0]
  #se ci sono errori parte da qui: too many values to unpack (expected 2), non riesce a trovare prob
  #trova le n emoticono più alte 
  emoji_ids = top_elements(prob, top_n)

  #converte questi numeri in emoticons
  emojis = map(lambda x: EMOJIS[x], emoji_ids)
  
  if return_emoji == False and return_prob == False:
    return None
  elif return_emoji == True and return_prob == False:
    return emoji.emojize(f"{sentence} {' '.join(emojis)}", use_aliases=True)
  elif return_emoji == True and return_prob == True:
    return emoji.emojize(f"{sentence} {' '.join(emojis)}", use_aliases=True), prob
  elif return_emoji == False and return_prob == True:
    return prob

#too many values to unpack (expected 2), per risolvere resettare e reinstallare, dipende da pack 
deepmojify('ciao come stai?', top_n=3, return_emoji=True, return_prob=False)

/content/torchMoji/torchmoji/model_def.py:159: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(self.embed.weight.data, a=-0.5, b=0.5)
/content/torchMoji/torchmoji/model_def.py:161: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(t)
/content/torchMoji/torchmoji/model_def.py:163: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
  nn.init.orthogonal(t)
/content/torchMoji/torchmoji/model_def.py:165: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(t, 0)
/content/torchMoji/torchmoji/model_def.py:167: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(self.output_layer[0].weight.data)
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1320: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.w

'ciao come stai? 💓 💛 ❤'

In [6]:
#too many values to unpack (expected 2), per risolvere resettare e reinstallare, dipende da pack 
deepmojify('ciao come stai?', top_n=3, return_emoji=False, return_prob=True)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1320: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


array([0.00394054, 0.00129295, 0.00197147, 0.00432513, 0.01371501,
       0.00560058, 0.00418637, 0.02010225, 0.07735906, 0.01391253,
       0.01317994, 0.00444024, 0.0065301 , 0.0008908 , 0.0036633 ,
       0.00902654, 0.03026676, 0.00181301, 0.03949212, 0.00103115,
       0.00485938, 0.00700757, 0.00463908, 0.02274275, 0.17968854,
       0.00277099, 0.00447128, 0.00739905, 0.02860775, 0.00278266,
       0.03876906, 0.0063387 , 0.00113546, 0.00621301, 0.00847516,
       0.00747591, 0.01020002, 0.00090959, 0.00828929, 0.00162419,
       0.00219347, 0.0071467 , 0.00454975, 0.0043973 , 0.00239056,
       0.00502038, 0.00712393, 0.13541125, 0.00248322, 0.02106217,
       0.02085136, 0.00176592, 0.00304266, 0.02627314, 0.01630236,
       0.00183573, 0.00255118, 0.00402961, 0.00483469, 0.02000473,
       0.02692209, 0.02151149, 0.00628529, 0.04087168], dtype=float32)

In [8]:
import pandas as pd

#converts a list to emoji
def emoji_dataset(list1, n_emoji=3, only_prob=False):
  emoji_list = [[x] for x in list1]

  for _ in range(len(list1)):
    for n_emo in range(1, n_emoji+1):
      if only_prob == False:
        emoji_list[_].append(deepmojify(list1[_], top_n=n_emoji, return_emoji=True, return_prob=False)[2*-n_emo+1])
      else:
        emoji_list[_].append(deepmojify(list1[_], top_n=1, return_emoji=False, return_prob=True))

  emoji_list = pd.DataFrame(emoji_list)
  return emoji_list

##Emotional Detector

In [9]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Projects/20200621_Emotion_Detector/Thanos.csv')
df = pd.DataFrame(df['tweet'])
df

,tweet
0,Ich würde es nicht wie #Thanos machen. Ich wür...
1,#lol #nerdy #avengersinfinitywar #johnwick #tu...
2,Happy Father’s Day all! 🧔🏻👨🏿👨🏽‍🦳🦝🦍Your a bette...
3,6／21お題「撃」\n撃攘\n#IRONMAN\n#THANOS\n#夜のフィギュア撮影会 ...
4,¿Y si el chasquido de Thanos hubiera ocurrido ...
...,...
5133,People picking sides with Tiger King are the s...
5134,A concept #digitalart - So Today was the day w...
5135,@MarvelStudios @Russo_Brothers I heard there i...
5136,Después del mal sabor de boca de #LaAlianzaDel...


In [8]:
def check_string(string1):
  if '#thanos' in string1:
    return 1
  elif 'Thanos' in string1:
    return 1
  elif 'thanos' in string1:
    return 1
  elif 'THANOS' in string1:
    return 1
  else:
    return 0

#cleaning the data
df['healthy'] = 0
df

#only conserve tweets with the right keywords
df['healthy'] = df['tweet'].apply(lambda x : check_string(x))
df

df = df.loc[df['healthy'] == 1]
df

,tweet,healthy
0,Ich würde es nicht wie #Thanos machen. Ich wür...,1
1,#lol #nerdy #avengersinfinitywar #johnwick #tu...,1
2,Happy Father’s Day all! 🧔🏻👨🏿👨🏽‍🦳🦝🦍Your a bette...,1
3,6／21お題「撃」\n撃攘\n#IRONMAN\n#THANOS\n#夜のフィギュア撮影会 ...,1
4,¿Y si el chasquido de Thanos hubiera ocurrido ...,1
...,...,...
5133,People picking sides with Tiger King are the s...,1
5134,A concept #digitalart - So Today was the day w...,1
5135,@MarvelStudios @Russo_Brothers I heard there i...,1
5136,Después del mal sabor de boca de #LaAlianzaDel...,1


In [9]:
df = df['tweet'].to_list()
df

['Ich würde es nicht wie #Thanos machen. Ich würde mir eine pick&choose-Funktion im Handschuh einbauen lassen.',
 '#lol #nerdy #avengersinfinitywar #johnwick #tumblr #nerd #russobrothers #avengers #infinitywar #thanos #thesnap #nerd #talknerdytome #TNTM  https://www.instagram.com/p/CBsnVejFSU65uiHx9EGnscpImlek4MMSR0PdkI0/?igshid=mea9d56ri9p8\xa0…',
 'Happy Father’s Day all! \U0001f9d4🏻👨🏿👨🏽\u200d\U0001f9b3\U0001f99d🦍Your a better Dad then #thanos\n#dadoftheyear #bestdadever #HappyFathersDay2020 \n\nEpisodes available here:\n🍏  https://spoti.fi/2Y42faQ\xa0\n#podnation #podcast #PodNation #podernfamily #movienight #whatshouldwewatch #quarentineandchill pic.twitter.com/XgqE6F1x7F',
 '6／21お題「撃」\n撃攘\n#IRONMAN\n#THANOS\n#夜のフィギュア撮影会 pic.twitter.com/MgffdcjT7m',
 '¿Y si el chasquido de Thanos hubiera ocurrido en la vida real? Mira este vídeo del canal QUE PASARÍA SÍ? y descúbrelo conmigo.\n\n  https://youtu.be/ImWr0au7CZk\xa0\n⬆️⬆️⬆️⬆️⬆️⬆️⬆️⬆️⬆️\n#THANOS #avengersendgame #AVENGERS #avengersinfi

In [10]:
#start converting text to emoji
df = emoji_dataset(df, 5)
df

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1320: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


,0,1,2,3,4,5
0,Ich würde es nicht wie #Thanos machen. Ich wür...,😢,😄,😓,😅,💓
1,#lol #nerdy #avengersinfinitywar #johnwick #tu...,😅,😉,😄,😎,😜
2,Happy Father’s Day all! 🧔🏻👨🏿👨🏽‍🦳🦝🦍Your a bette...,😊,💪,👍,😄,😉
3,6／21お題「撃」\n撃攘\n#IRONMAN\n#THANOS\n#夜のフィギュア撮影会 ...,💓,✌,👍,🎶,🎵
4,¿Y si el chasquido de Thanos hubiera ocurrido ...,👊,😓,😡,💓,😠
...,...,...,...,...,...,...
5132,People picking sides with Tiger King are the s...,👍,😐,😕,😠,😡
5133,A concept #digitalart - So Today was the day w...,😕,😑,😠,👍,😡
5134,@MarvelStudios @Russo_Brothers I heard there i...,😊,😄,😜,👍,😉
5135,Después del mal sabor de boca de #LaAlianzaDel...,😠,😷,💛,💓,☺


In [13]:
#export dataset
df.to_csv('Thanos_.csv')

##Analyze data (experimental)

In [42]:
final = df.groupby([1, 2, 3, 4, 5, 0], group_keys=True, squeeze=False, sort=True)
final = pd.DataFrame(final)
final

,0,1
0,"(☺, ✌, ❤, 💛, 💓, #AvengersEndgame #Thanos #Marv...",...
1,"(☺, ✌, ❤, 💛, 💓, #DíaNacionalDelContagio #Duque...",...
2,"(☺, ✌, 👍, 👊, 💪, Iron Man's Best Fighting Moves...",...
3,"(☺, ✌, 👍, 👊, 💪, Iron Man's Best Fighting Moves...",...
4,"(☺, ✌, 👍, 💛, 💓, #infosec #informationsecurity ...",...
...,...,...
5037,"(🙏, 😡, 😠, 😢, 🎶, Omg yesssssss!!!! #MARVEL #iro...",...
5038,"(🙏, 😢, 🎶, 👊, 💪, The last battle man #marvelall...",...
5039,"(🙏, 😢, 👊, 👏, 👍, 🍿📺🍿 NEW VIDEO 🍿📺🍿\n\nDeleted A...",...
5040,"(🙏, 😢, 👍, 👏, ✨, Recently, an unknown deleted s...",...


In [43]:
final = df.groupby([1, 2, 3, 4, 5], group_keys=True, squeeze=True, sort=True).size().sort_values(ascending=False)
final = pd.DataFrame(final)
final

0
1 2 3 4 5    
🙏 🎵 👏 👍 🎶  80
😍 💙 💓 💛 ❤  75
🙏 👏 🎵 👍 🎶  46
😊 🙏 💪 😉 👍  40
😍 💪 💙 💛 💓  23
...        ..
😉 😄 💪 😜 👍   1
      😎 👍   1
    💛 😜 💓   1
      💓 ☺   1
💯 🎶 💪 😈 👊   1

[4090 rows x 1 columns]